In [66]:
%load_ext autoreload
%autoreload 2

import torch
# from PolySym import Regressor, Operators
from polysym.torch_operators import Operators
from polysym.regressor import Configurator
from polysym.model import PolySymModel
from polysym.evaluation import r2

n_obs = 1000

X3d = torch.zeros((n_obs, 2, 100))
X2d = torch.zeros((n_obs, 1))
y1d = torch.zeros(n_obs)
y2d = torch.zeros(n_obs, 100)

for obs in range(n_obs):

    start, end = torch.rand(2) * 100

    x1 = torch.linspace(start, end, 100)
    x2 = torch.cos(torch.linspace(start, end, 100))
    b = torch.randint(low=-10, high=10, size=(1, 1))

    y = (b + x1 + (x1 * x2)) * 14.31
    # expr=binary_add(binary_add(v0, x0), binary_mul(v1, v0))

    X3d[obs, 0] = x1
    X3d[obs, 1] = x2
    X2d[obs] = b
    y1d[obs] = torch.mean(y).item()
    y2d[obs] = y

operators = Operators(['add', 'sub', 'mul', 'div', 'neg'])

model = PolySymModel(X3d=X3d,
                        X2d=X2d,
                        y=y2d,
                        operators=operators,
                        min_complexity=3,
                        max_complexity=5,
                        pop_size=300,
                        stopping_criterion=.9,
                        max_iter=2000,
                        fitness_fn = r2,
                        fitness_obj = 1,
                        seed=42,
                        verbose=1,
                        workers=-1)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
from polysym.funsearch import FunSearch


fs = FunSearch(pset=model.pset,
               ollama_model='deepseek-r1:1.5b')


In [97]:
from ollama import Client

fs._call_ollama("What's the capital of the moon?", temperature=.1)

'\n\nI am sorry, I cannot answer that question.'

In [92]:
import numpy as np

pop = model.toolbox.population(n=20)
fitnesses = np.random.random(20).tolist()

In [93]:
prompt = fs._build_prompt(pop, fitnesses, 'minimization', 30)

print(prompt)

You are a symbolic‑regression assistant.  I will give you a list of existing expressions (in DEAP’s prefix notation) together with their fitnesses.  Your job is to propose a specified number of new candidate expressions that

• use only these tokens:
    – Variables:   x0, x1, …      (2D inputs)
                   v0, v1, …      (3D inputs)
    – Unary ops:  unary_add, unary_sub, unary_mul, unary_div, unary_neg,
                   unary_sin, unary_cos, unary_tan, unary_exp, unary_log
    – Binary ops: binary_add, binary_sub, binary_mul, binary_div, binary_pow
    – Numeric constants: any real in decimal form (e.g. 3.14, –0.5)

• are syntactically valid DEAP prefix expressions (PrimitiveTree.from_string must accept them)
• contain only the allowed tokens and whitespace—no commas, parentheses, bullets or commentary
• appear one expression per line, nothing else
• complexities of expression must vary and be as diverse as in the following context
• YOU MAY ONLY OUTPUT NEW EXPRESSIONS THAT 

In [106]:
npop, response = fs.propose(pop, fitnesses, 'minimization', 10, temperature=.1)

Got response
Failed to build tree with line Here are 10 valid expressions in prefix notation using the allowed operators and terminals:


In [107]:
print(len(npop))

print(response)

10


Here are 10 valid expressions in prefix notation using the allowed operators and terminals:

1. unary_neg(x0)
2. binary_add(x0, v0)
3. unary_neg(binary_sub(v1, v0))
4. binary_mul(unary_neg(x0), v1)
5. binary_div(binary_mul(x0, v0), v1)
6. unary_neg(binary_div(x0, v1))
7. binary_add(unary_neg(x0), unary_neg(v1))
8. binary_mul(unary_neg(v0), x0)
9. binary_div(binary_sub(v1, v0), x0)
10. unary_neg(binary_div(x0, v1))


In [108]:
for i in npop:
    print(str(i))

unary_neg(x0)
binary_add(x0, v0)
unary_neg(binary_sub(v1, v0))
binary_mul(unary_neg(x0), v1)
binary_div(binary_mul(x0, v0), v1)
unary_neg(binary_div(x0, v1))
binary_add(unary_neg(x0), unary_neg(v1))
binary_mul(unary_neg(v0), x0)
binary_div(binary_sub(v1, v0), x0)
unary_neg(binary_div(x0, v1))


In [27]:
for i in pop:
    print(str(i))

unary_neg(unary_neg(binary_sub(binary_add(binary_add(v0, x0), binary_div(x0, -93.64346410364328)), unary_neg(binary_add(v0, x0)))))
binary_add(binary_div(binary_mul(binary_add(binary_mul(v1, x0), binary_add(-31.949896696401623, v0)), binary_add(binary_sub(x0, x0), binary_mul(x0, v1))), binary_sub(binary_div(binary_sub(x0, 7.245618290940143), binary_mul(x0, v1)), binary_div(binary_sub(v0, x0), unary_neg(v0)))), binary_sub(unary_neg(binary_add(unary_neg(-44.40527937798158), binary_sub(v0, v1))), binary_sub(binary_add(binary_sub(x0, v0), binary_div(v0, v0)), binary_mul(binary_mul(v1, v0), binary_sub(x0, v0)))))
binary_sub(binary_mul(binary_sub(x0, v0), binary_div(v1, v0)), binary_mul(binary_mul(-71.42568191158747, v0), binary_add(v1, 79.56457672049538)))
binary_sub(binary_add(binary_add(binary_div(-81.81811756524122, x0), unary_neg(v0)), binary_add(binary_mul(x0, -23.676142698692644), binary_mul(v1, x0))), unary_neg(binary_div(binary_sub(v1, x0), binary_sub(-68.36844913106606, x0))))
bina

[autoreload of polysym.funsearch failed: Traceback (most recent call last):
  File "/Users/nizarmichaud/VSCodeProjects/PolySym/venv/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 283, in check
    superreload(m, reload, self.old_objects)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nizarmichaud/VSCodeProjects/PolySym/venv/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 483, in superreload
    module = reload(module)
  File "/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/importlib/__init__.py", line 129, in reload
    _bootstrap._exec(spec, module)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 1018, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1156, in get_code
  File "<frozen importlib._bootstrap_external>", line 1086, in source_to_code
  File "<frozen i

In [18]:
import sympy as sp

sp.simplify(sp.sympify(str(pop[0])))

unary_neg(unary_neg(binary_sub(binary_add(binary_add(ARG1, v1), unary_neg(v1)), binary_div(unary_neg(v0), binary_mul(ARG0, ARG0)))))

In [29]:
str(pop[-1])

'binary_sub(binary_div(unary_neg(binary_div(unary_neg(v1), binary_div(32.30426813747118, x0))), binary_div(binary_sub(unary_neg(64.46618361801163), unary_neg(92.15751344291573)), binary_mul(binary_add(-64.77260778278813, v1), binary_div(-7.025245673098439, v1)))), binary_sub(binary_add(unary_neg(binary_sub(-51.227312161808804, 13.969928935664441)), binary_mul(binary_sub(x0, 70.19056726249181), binary_add(-57.576029964052196, v1))), binary_sub(binary_sub(binary_div(v1, x0), binary_div(v0, x0)), binary_add(binary_mul(11.035242316372162, v0), binary_mul(-10.365090082685839, x0)))))'